### To change regarding the simulation we want

In [1]:
#Dataset to use Enron email or Hep
datasetToUse = "Enron" #Enron or Hep
#Approximate size of the community we want to have rumor spread from
#We will pick the community with the closest size if we don't find it exactly
wantedSize = 2600
#%age of nodes in the community that will spread rumors
rumorProportion = 0.1

nbOfSimulations = 20

### Run all

In [2]:
import community as cmy
import networkx as nx

from collections import defaultdict

import numpy as np
import random

import operator

In [3]:
class Graph: 

	# Constructor 
	def __init__(self): 
		# default dictionary to store graph 
		self.graph = defaultdict(list) 
		#to count number of nodes
		self.control=set()
	# function to add an edge to graph 
	def addEdge(self,u,v):
		#just for testing...should be removed...nodes will be counted from file
		if u not in self.control:
			self.control.add(u)
		if v not in self.control:
			self.control.add(v)
		self.graph[u].append(v) 

def BFS(graph, s,community):
    edge_ends=[] #our desired output
    visited=[]
    #number of nodes should not be fixed
    visited = dict()
    
    for i in g.control:
        visited[i] = False
    
    #Number of nodes in grapth
    #print(len(graph.control))
    #get the graph per si
    graph=graph.graph
    # Create a queue for BFS 
    queue = [] 

    # Mark the source node as 
    # visited and enqueue it 
    for i in s:
        queue.append(i) 
        visited[i] = True

    while queue: 

        # Dequeue a vertex from queue
        s = queue.pop(0) 

        # Get all adjacent vertices of the 
        # dequeued vertex s. If a adjacent is from the community 
        # and has not been visited, then mark it 
        # visited and enqueue it otherwise just mark it as visited and add to edges vetor
        for i in graph[s]:
            if i in community:
                if visited[i] == False:
                    queue.append(i) 
                    visited[i] = True
            else:
                if visited[i] == False:
                    visited[i] = True
                    edge_ends.append(i)
    return edge_ends


#basic function to find all paths between 2 nodes	
def find_all_paths(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            newpaths = find_all_paths(graph, node, end, path)
            for newpath in newpaths:
                paths.append(newpath)
    return paths


In [4]:
def findParents(graph, node):
    listOfParents = []
    for i in graph.keys():
        if node in graph[i]:
            listOfParents.append(i)
    return listOfParents

def backwardTree(graph, bridgeEnd, rumors):
    tree = [[bridgeEnd]]
    visited = [bridgeEnd]
    findOneRumor = False
    while not findOneRumor:
        nbElement = len(tree)
        for i in range(nbElement):
            branch = tree.pop(0)
            listOfParents = findParents(g.graph, branch[-1])
            for parent in listOfParents:
                if parent not in visited:
                    tree.append(branch + [parent])
                    visited.append(parent)
                    findOneRumor = findOneRumor or (parent in rumors)
    return visited, tree

def protectorCandidateSets(tree, rumors):
    candidateSet = []
    for branch in tree:
        candidateSet += branch
    candidateSet = set(candidateSet).difference(set(rumors))
    return candidateSet

def backwardTree2(graph, bridgeEnd, rumors):
    fifoQ = [bridgeEnd]
    visited = [bridgeEnd]
    findOneRumor = False
    while not findOneRumor and len(fifoQ) != 0:
        branch = fifoQ.pop(0)
        listOfParents = findParents(g.graph, branch)
        for parent in listOfParents:
            if parent not in visited:
                fifoQ.append(parent)
                visited.append(parent)
                findOneRumor = findOneRumor or (parent in rumors)
    return visited



In [5]:
g = Graph() 
if datasetToUse == "Enron":
    emailEnron = np.loadtxt("Email-Enron.txt")
    for i in emailEnron:
        g.addEdge(int(i[0]),int(i[1]))
elif datasetToUse == "Hep":
    hepPh = np.loadtxt("CA-HepPh.txt") 
    for i in hepPh:
        g.addEdge(int(i[0]),int(i[1]))

In [6]:
def getLouvainCommunities(g): 
    G=nx.Graph(g.graph)
    communities = cmy.best_partition(G)
    c = [ [] for i in range(len(set(communities.values()))) ]
    for i in communities.keys():
        c[communities[i]].append(i)
    return c

#Getting communities set
community = getLouvainCommunities(g)

In [7]:
nbOfProtector = []

In [ ]:
for i in range(nbOfSimulations):
    print(i+1)
    index = np.argmin([abs(len(community[j]) - wantedSize) for j in range(len(community))])

    #here for simplicity i use only one rumor node (easy to fix)
    Srumor=random.choices(community[index], k=int(rumorProportion * len(community[index])))

    #run the customized BFS function on the graph

    bridgeEnds = BFS(g, Srumor, community[index])

    candidates = dict()
    tmp = 0

    for bridge in bridgeEnds:
        tmp += 1
        visited = backwardTree2(g.graph, bridge, Srumor)
        pCandidates = set(visited).difference(set(Srumor))
        #pCandidates = protectorCandidateSets(visited, Srumor)
        for i in pCandidates:
            if i not in candidates.keys():
                candidates[i] = []
            candidates[i].append(bridge)

    Sp = []
    PB = []

    maxProtection = 0
    while len(PB) != len(bridgeEnds):
        choosenCandidate = 0
        for i in candidates.keys():
            protection = len(set(PB + candidates[i]))
            if protection > maxProtection:
                maxProtection = protection
                choosenCandidate = i
        Sp.append(choosenCandidate)
        PB = list(set(PB + candidates[choosenCandidate]))
        candidates.pop(choosenCandidate)

    outDegree = dict()
    for i in g.graph.keys():
        outDegree[i] = len(set(g.graph[i]))  #.intersection(bridgeEnds))
    SpMaxDegree = []
    PBMaxDegree = []
    while len(PBMaxDegree) != len(bridgeEnds):
        newProtector = max(outDegree.items(), key=operator.itemgetter(1))[0]
        SpMaxDegree.append(newProtector)
        protectedBridges = set(g.graph[newProtector] + [newProtector]).intersection(bridgeEnds)
        PBMaxDegree = list(set(PBMaxDegree + list(protectedBridges)))
        #print(len(PBMaxDegree), newProtector, outDegree[newProtector])
        del outDegree[newProtector]

    SpProximity = []
    for r in Srumor:
        SpProximity += g.graph[r]

    SpProximity = list(set(SpProximity))
    
    nbOfProtector.append([len(g.control), wantedSize, rumorProportion, len(Sp), len(SpMaxDegree), len(SpProximity)])

1
2
3
4
5
6
7
8


### Results

In [ ]:
print("Nb of simulation:", nbOfSimulations)
print("Nb of nodes:", len(g.control))
print("Community size:", len(community[index]))
print("R:", rumorProportion,"%")
SCBG = 0
Proximity = 0
MaxDegree = 0
for i in nbOfProtector:
    SCBG += i[3]
    Proximity += i[4]
    MaxDegree += i[5]

SCBG /= nbOfSimulations
Proximity /= nbOfSimulations
MaxDegree /= nbOfSimulations
print("SCBG:", SCBG)
print("Proximity:", Proximity)
print("MaxDegree:", MaxDegree)